# 한국어 텍스트 분류 노트북

이 Jupyter Notebook은 `konlpy` 라이브러리를 사용하여 한국어 텍스트 데이터를 전처리하고, `scikit-learn`을 이용해 텍스트 분류 모델을 학습하는 과정을 보여줍니다.

---

## 1. 라이브러리 임포트

필요한 라이브러리들을 임포트합니다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import re # 정규식 처리 라이브러리
from konlpy.tag import Okt # 현재 가장 많이 사용하는 형태소 분리 알고리즘
from sklearn.linear_model import LogisticRegression

---

## 2. 데이터 로드 및 확인

`aclImdb/ratings_train.txt` 파일을 로드하고 데이터의 처음 몇 행을 확인합니다. 이 데이터는 영화 리뷰와 해당 리뷰의 긍정/부정 레이블을 포함하고 있습니다.

In [ ]:
# 파일을 읽는다 - 구분자가 탭키다
# keep_default_na - NaN 값을 None으로 바꾼다
df_train = pd.read_csv("./data/aclImdb/ratings_train.txt", delimiter="\t", keep_default_na=False)
print("데이터프레임 상위 5행:")
print( df_train.head() )

# 'document' 열과 'label' 열을 추출한다
text_train, y_train = df_train["document"].values, df_train["label"].values
print("\n텍스트 데이터 샘플 (상위 3개):")
print( text_train[:3])
print("\n레이블 데이터 샘플 (상위 3개):")
print( y_train[:3])

---

## 3. 텍스트 전처리 및 토큰화

`Okt` 형태소 분석기를 사용하여 텍스트를 전처리하고 토큰화하는 함수를 정의합니다. 불필요한 특수문자를 제거하고, 불용어(stop words)를 필터링하며, 길이가 2 미만인 단어는 제외합니다.

In [ ]:
okt = Okt()
stop_words = ["아", "..", "?", "있는", "은", "는", "이", "가", "하", "아", "것", "들", "의", "한", "에", "와", "과", "을", "를", "으로", "로", "에서", "에게", "뿐", "도", "만", "하여", "에게", "로", "으로", "까지", "부터", "이다", "다"] # 빼고 싶은 단어들 리스트에 추가

def okt_tokenizer(text):
    # 특수문자를 제거하기 (숫자와 영어 등을 제거 - 공백으로 대체)
    # 정규식 패턴을 사용해서 문자 바꿔치기. 한글 초성, 중성, 종성 범위 유지
    text = re.sub(r"[^가-힣\s]", "", text)
    temp = okt.morphs(text, stem=True) # 형태소 분리 및 어간 추출
    # 제거할거 있으면 제거시켜서 보내기, 불필요한 단어나 한글자는 삭제시키고 나머지만
    temp = [word for word in temp if word not in stop_words and len(word) >= 2]
    return temp

print("토큰화된 텍스트 샘플 (상위 10개):")
for i in range(0, 10):
    print( okt_tokenizer(text_train[i] ))

---

## 4. 특성 벡터화

`CountVectorizer`를 사용하여 텍스트 데이터를 수치형 특성 벡터로 변환합니다. 이때, 위에서 정의한 `okt_tokenizer` 함수를 `tokenizer` 매개변수로 전달하여 한국어 텍스트에 맞는 토큰화를 수행합니다.

In [ ]:
# CountVectorizer의 tokenizer 매개변수에 우리가 만든 토큰나이저를 사용한다.
# 한글 토큰나이저로 바꿔치기를 한다. 경고는 무시해도 된다.
vect = CountVectorizer(tokenizer=okt_tokenizer).fit(text_train)

feature_names = vect.get_feature_names_out()
print("\n특성의 개수 (단어 사전 크기):", len(feature_names))
print("특성 이름 샘플 (상위 20개):")
print(feature_names[:20])

# 훈련 데이터를 특성 벡터로 변환한다
X_train = vect.transform(text_train)
print("\n변환된 훈련 데이터의 형태:", X_train.shape)

---

## 5. 모델 학습 및 평가

로지스틱 회귀 모델을 초기화하고, 벡터화된 훈련 데이터를 사용하여 모델을 학습시킵니다. 학습된 모델의 훈련 데이터에 대한 정확도를 평가합니다.

In [ ]:
# 로지스틱 회귀 모델을 초기화한다
model = LogisticRegression(solver='liblinear', random_state=0) # solver='liblinear'는 작은 데이터셋에 적합하며, random_state는 결과 재현성을 위해 설정

# 모델을 훈련 데이터에 맞게 학습시킨다
model.fit(X_train, y_train)

# 훈련 데이터에 대한 모델의 정확도를 출력한다
print("\n훈련 데이터 정확도:", model.score(X_train, y_train))